In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from tqdm.notebook import trange, tqdm
import warnings
import time


warnings.filterwarnings('ignore')

In [2]:
import pickle
with open("./../screener/optionable_ticker_for_val", "rb") as fp:   # Unpickling
    lst = pickle.load(fp)

In [3]:
def options_chain(symbol):


    exps =set([])
    try_cnt = 0
    while (len(exps) == 0) and (try_cnt < 3):
        tk = yf.Ticker(symbol)
        try:
            exps = tk.options
        except:
            print('exps = tk.options has error')
        try_cnt += 1
        if (not tk) or (len(exps) == 0):
            time.sleep(10)
    
    # if not able to fetch expire dates, return empty dataset
    if len(exps) == 0:
        print(f'cannot fetch option expire dates for {symbol}')
        return pd.DataFrame({'symbol':[symbol]})
    
    # Get options for each expiration
    options = pd.DataFrame()
    
    for e in exps:
        opt = 0
        try2_cnt = 0
        while try2_cnt < 3:
            try:
                opt = tk.option_chain(e)
                opt = pd.concat([opt.calls, opt.puts], ignore_index=True)
                opt['expirationDate'] = e
                options = pd.concat([options, opt], ignore_index=True)
                break
            except:
                try2_cnt += 1

    # if all expire dates' data pull are not successful, return empty dataset
    if len(options) == 0:
        print(f'cannot fetch option chain for {symbol}')
        return pd.DataFrame({'symbol':[symbol]})

    # Bizarre error in yfinance that gives the wrong expiration date
    # Add 1 day to get the correct expiration date
    options['expirationDate'] = pd.to_datetime(options['expirationDate'])
    options['dte'] = (options['expirationDate'] - datetime.datetime.today()).dt.days / 365
    
    # Boolean column if the option is a CALL
    options['CALL'] = options['contractSymbol'].str[4:].apply(
        lambda x: 1 if "C" in x else 0)
    
    options[['bid', 'ask', 'strike']] = options[['bid', 'ask', 'strike']].apply(pd.to_numeric)
    options['mark'] = (options['bid'] + options['ask']) / 2 # Calculate the midpoint of the bid-ask
    options['symbol'] = symbol
    options['lastTradeDate'] = options.lastTradeDate.dt.tz_convert('US/Eastern')
    
    # Drop unnecessary and meaningless columns
#     options = options.drop(columns = ['contractSize', 'currency', 'change', 'percentChange', 'lastTradeDate', 'lastPrice'])

    return options

In [4]:
%%time
all_options = []
for t in tqdm(lst):
#     try:
    print(t)
    oc = options_chain(t)
    all_options.append(oc)
#     except:
#         print(f'not able to get data for {t}')
#         continue

  0%|          | 0/1781 [00:00<?, ?it/s]

A
AA
AAL
AAON
AAP
AAPL
AAWW
AB
ABB
ABBV
ABC
ABCB
ABCL
ABCM
ABG
ABM
ABNB
ABR
ABT
ACA
ACAD
ACDC
ACGL
ACHC
ACI
ACIW
ACLS
ACM
ACN
ACT
ADBE
ADC
ADI
ADM
ADNT
ADP
ADSK
ADT
AEE
AEG
AEIS
AEL
AEM
AEO
AEP
AER
AES
AFG
AFL
AFRM
AG
AGCO
AGI
AGL
AGNC
AGO
AGTI
AGYS
AHCO
AIG
AIMC
AIN
AIRC
AIT
AIZ
AJG
AJRD
AKAM
AKRO
AL
ALB
ALC
ALGM
ALGN
ALHC
ALIT
ALK
ALKS
ALL
ALLE
ALLY
ALNY
ALRM
ALSN
ALTR
ALV
AM
AMAT
AMBA
AMBP
AMC
AMCR
AMD
AME
AMED
AMG
AMGN
AMH
AMKR
AMLX
AMN
AMP
AMR
AMRC
AMT
AMX
AMZN
AN
ANET
ANSS
AON
AOS
APA
APAM
APD
APG
APH
APLE
APLS
APO
APP
APPF
APPN
APTV
AQN
AQUA
AR
ARCC
ARCH
ARE
ARES
ARGX
ARLP
ARMK
ARNC
ARRY
ARW
ARWR
ASAI
ASAN
ASB
ASGN
ASH
ASML
ASND
ASO
ASX
ATCO
ATHM
ATI
ATKR
ATO
ATR
ATVI
AU
AUB
AUY
AVA
AVAV
AVB
AVDX
AVGO
AVNT
AVT
AVTR
AVY
AWI
AWK
AWR
AX
AXNX
AXON
AXP
AXS
AXSM
AXTA
AY
AYI
AYX
AZEK
AZN
AZO
AZPN
AZTA
B
BA
BABA
BAC
BAH
BALL
BANR
BAP
BAX
BBVA
cannot fetch option expire dates for BBVA
BBWI
BBY
BC
BCC
BCE
BCH
BCO
BCS
BDC
BDX
BE
BEAM
BECN
BEKE
BEN
BEP
BEPC
BERY
BF-B
BFAM
BG
BGNE
BHC
BHF
BH

TU
TV
TW
TWKS
TWLO
TWNK
TX
TXG
TXN
TXRH
TXT
TYL
U
UA
UAA
UAL
UBER
UBS
UDR
UFPI
UGI
UHAL
UHS
UL
ULCC
ULTA
UMBF
UMC
UMPQ
UNFI
UNH
UNM
UNP
UNVR
UPS
URBN
URI
USAC
USB
USFD
USM
UTHR
UTZ
V
VAC
VAL
VALE
VC
VEEV
VET
VFC
VIAV
VICI
VICR
VIPS
VIR
cannot fetch option expire dates for VIR
VIRT
VIV
VLO
VLY
VMC
VMI
VMW
VNO
VNOM
VNT
VOD
VOX
VOYA
VRNS
VRNT
VRRM
VRSK
VRSN
VRT
VRTX
VSAT
VSCO
VSH
VST
VTR
VTRS
VVNT
VVV
VZ
W
WAB
WAFD
WAL
WAT
WB
WBA
WBD
WBS
WCC
WCN
WD
WDAY
WDC
WDS
WEBR
WEC
WELL
WEN
WERN
WES
WEX
WFC
WFG
WFRD
WH
WHR
WING
WIRE
WIX
WK
WLK
WLY
WM
WMB
WMG
cannot fetch option expire dates for WMG
WMS
WMT
WNS
cannot fetch option expire dates for WNS
WOLF
WOOF
WOR
WPC
WPM
WPP
WRB
WRK
WSC
WSFS
WSM
WSO
WST
WTFC
WTRG
WTS
WTW
WU
WWD
WWE
WY
WYNN
X
XEL
XENE
XM
XOM
XP
XPEL
XPEV
XPO
XRAY
XRX
XYL
YELP
YETI
YMM
YOU
YPF
YUM
YUMC
YY
Z
ZBH
ZBRA
ZD
ZG
ZI
ZIM
ZION
ZIP
ZLAB
ZM
ZS
ZTO
ZTS
ZWS
Wall time: 14h 14min 7s


In [5]:
df = pd.concat(all_options,ignore_index=True)

In [6]:
df.shape

(327369, 19)

In [7]:
df['inTheMoney'] = df.inTheMoney.astype('bool')

In [8]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2023-02-11


In [9]:
today = date(2023,2,10)
str(today)

'2023-02-10'

In [10]:
df.to_parquet(f"./saved_data/options_data_{str(today).replace('-','')}.gzip",
              compression='gzip')  